In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "14723717494252426396", 2, "feature")

In [4]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [5]:
y = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "11385547551736664765", 2, "feature")

## Vecchia funzione

In [6]:
function box(model)
	if isa(model,Matrix)
		return nothing
	elseif isa(model,Lar.Struct)
		listOfModels = Lar.evalStruct(model)
		#dim = checkStruct(listOfModels)
		if listOfModels == []
			return model.box
		else
			theMin,theMax = box(listOfModels[1])
			for theModel in listOfModels[2:end]
				modelMin,modelMax = box(theModel)
				for (k,val) in enumerate(modelMin)
					if val < theMin[k]
						theMin[k]=val
					end
				end
				for (k,val) in enumerate(modelMax)
					if val > theMax[k]
						theMax[k]=val
					end
				end
			end
		end
		return [theMin,theMax]

	elseif (isa(model,Tuple) ||isa(model,Array))&& (length(model)>=2)
		V = model[1]
		theMin = minimum(V, dims=2)
		theMax = maximum(V, dims=2)
	end

	return [theMin,theMax]
end

box (generic function with 1 method)

In [7]:
@btime box(x)

  9.801 μs (53 allocations: 2.72 KiB)


2-element Array{Array{Float64,2},1}:
 [0.0; 0.0]
 [1.0; 1.0]

In [8]:
@btime box(y)

  26.100 μs (118 allocations: 5.55 KiB)


2-element Array{Array{Float64,2},1}:
 [-0.5; -0.5]
 [0.5; 0.5]

## Nuove funzioni

In [9]:
function evalBox(listOfModels)
	theMin,theMax = box(listOfModels[1])
	for theModel in listOfModels[2:end]
		modelMin,modelMax= box(theModel)
		for (k,val) in enumerate(modelMin)
			if val < theMin[k]
				theMin[k]=val
			end
		end
		for (k,val) in enumerate(modelMax)
			if val > theMax[k]
				theMax[k]=val
			end
		end
	end
	return theMin,theMax
end

evalBox (generic function with 1 method)

In [10]:
function box(model)
	if isa(model,Matrix)
		return nothing
	elseif isa(model,Lar.Struct)
		listOfModels = Lar.evalStruct(model)
		#dim = checkStruct(listOfModels)
		if listOfModels == []
			return model.box
		else
			theMin,theMax = evalBox(listOfModels)
		end
        
		return [theMin,theMax]

	elseif (isa(model,Tuple) || isa(model,Array)) && (length(model)>=2)
		V = model[1]
		theMin = minimum(V, dims=2)
		theMax = maximum(V, dims=2)
	end
    
	return [theMin,theMax]
end

box (generic function with 1 method)

In [11]:
@btime box(x)

  10.499 μs (53 allocations: 2.72 KiB)


2-element Array{Array{Float64,2},1}:
 [0.0; 0.0]
 [1.0; 1.0]

In [12]:
@btime box(y)

  15.700 μs (116 allocations: 5.48 KiB)


2-element Array{Array{Float64,2},1}:
 [-0.5; -0.5]
 [0.5; 0.5]

In [13]:
@benchmark box(y)

BenchmarkTools.Trial: 
  memory estimate:  5.48 KiB
  allocs estimate:  116
  --------------
  minimum time:     16.899 μs (0.00% GC)
  median time:      50.101 μs (0.00% GC)
  mean time:        49.468 μs (1.85% GC)
  maximum time:     9.265 ms (98.53% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [14]:
@code_warntype box(y)

Variables
  #self#::Core.Compiler.Const(box, false)
  model::LinearAlgebraicRepresentation.Struct
  @_3::Int64
  listOfModels::Any
  V::Union{}
  theMin::Any
  theMax::Any
  @_8::Bool
  @_9::Bool

Body::Any
1 ──       Core.NewvarNode(:(@_3))
│          Core.NewvarNode(:(listOfModels))
│          Core.NewvarNode(:(V))
│          Core.NewvarNode(:(theMin))
│          Core.NewvarNode(:(theMax))
│    %6  = (model isa Main.Matrix)::Core.Compiler.Const(false, false)
└───       goto #3 if not %6
2 ──       Core.Compiler.Const(:(return Main.nothing), false)
3 ┄─ %9  = Base.getproperty(Main.Lar, :Struct)::Any
│    %10 = (model isa %9)::Bool
└───       goto #7 if not %10
4 ── %12 = Base.getproperty(Main.Lar, :evalStruct)::Any
│          (listOfModels = (%12)(model))
│    %14 = listOfModels::Any
│    %15 = Base.vect()::Array{Any,1}
│    %16 = (%14 == %15)::Union{Missing, Bool}
└───       goto #6 if not %16
5 ── %18 = Base.getproperty(model, :box)::Any
└───       return %18
6 ── %20 = Main.evalBox

## Modifiche